In [2]:
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### This code will produce arbitrary samples of binary deep eutectic solvents, taking the minimum and maximum mole fractions of each component, number of samples desired to make, and number of trials for the k-means clustering (should be much greater than the number of samples desired to produce). This can easily be modified for ternary deep eutectic solvents. 

In [3]:
def Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array([min_QAS, min_HBD])
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array([max_QAS, max_HBD])

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)



## Generalized

In [4]:
def Gen_Binary_DES_Generator(min_comps, max_comps, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array(min_comps)
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array(max_comps)

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)

### Generate binary DES with lower mole fractions of 0.2, 0.3 (QAS, HBD) and upper mole fractions of 0.7, 0.8 (QAS, HBD). Produce 96 samples with 192 trials for the clustering.

In [15]:
DES_molfrac= Binary_DES_Generator(0.2, 0.3, 0.7, 0.8, 96, 192)

 3840 feasible DES samples generated, clustered into 96 samples


### May also save the output as a variable. Note you will get different mole fractions each time so be sure to keep track if you need them.

In [6]:
A = Binary_DES_Generator(0.3, 0.3, 0.7, 0.8, 96, 192)
# A 

 3798 feasible DES samples generated, clustered into 96 samples


### The next function will combine the previous function with code to convert mole fractions to a list of lists of volumes that can be directly inputed into code for sample preparation in a pipetting robot (OT-2).

In [7]:
def convert_mole_fractions_to_volumes(stock_QAS, stock_HBD, min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    DES_mole_fractions = Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials)
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 125  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])  
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [14]:
    DES_molfrac = Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials)


NameError: name 'min_QAS' is not defined

## Generalized conversion to volumes

In [16]:
len(DES_molfrac)

96

In [21]:
samples = 10
volume = 150 
stock = [2,4]
number = len(stock)


size = (samples, number)
finvol = np.zeros(size)

stock_QAS = 2
stock_HBD = 4
DES_mole_fractions = Binary_DES_Generator(.2, .2, .8, .8, samples, 200)

count = 0
for row in DES_mole_fractions: 
    def f(x):
        for i in range(number):
            total = []
            place = stock[i]*x[i]
            total.append(place)
        total = sum(total)
        y = np.zeros(np.size(x))                                                                           
        y[0] = x[0] + x[1] - volume                                                                   
        for i in range(number-1):
            y[i+1] = (((stock[i])*x[i])/(total)) - row[i]
        return y
    x0 = np.array([100.0, 100.0, 100.0])  
    x = fsolve(f, x0)
    for i in range(number):
        finvol[count, i] = x[i]
    count = count + 1

print("finvol is", finvol)
number = len(finvol[0])
print(number)

 4000 feasible DES samples generated, clustered into 10 samples
finvol is [[73.60190978 76.39809022]
 [86.39044416 63.60955584]
 [59.16303714 90.83696286]
 [80.30776567 69.69223433]
 [65.03933234 84.96066766]
 [69.46016573 80.53983427]
 [89.56735151 60.43264849]
 [51.32643411 98.67356589]
 [83.40563463 66.59436537]
 [77.08195637 72.91804363]]
2


### Generalized

## Converting to Opentrons copyable

In [22]:
sepvol = []

for i in range(number):
    string = "comp" + str(i)
    string = []
    for row in finvol:
        hold = row[i]
        string.append(hold)
    sepvol.append(string)

sepvol.type()

AttributeError: 'list' object has no attribute 'type'

### Alternatively, you could use this next function if you already have a list you want to convert (as before when we saved the output of the des mole fractions)

In [11]:
def convert_mole_fractions_to_volumes_2(DES_mole_fractions, stock_QAS, stock_HBD):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 150  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [12]:
convert_mole_fractions_to_volumes_2(A, 3, 2)

[[50.37813191068493,
  69.8743078602248,
  38.8337827761865,
  58.93897614454336,
  80.48455343512126,
  44.15645306941992,
  63.04142567854789,
  54.43997097718037,
  84.64882277069829,
  74.94087560275959,
  34.90974621258966,
  47.41323894896303,
  66.05067004968399,
  41.39644819398069,
  56.330013058789774,
  77.5871318135321,
  61.211996595146076,
  52.1597834935777,
  89.3508409827377,
  36.999788552109294,
  67.85769672228446,
  48.17121583667832,
  72.92446587921062,
  45.07251170141245,
  46.36132502314877,
  64.39463694552583,
  57.466482270832905,
  42.94560134923387,
  52.967361461769855,
  81.78623271753477,
  40.31645960367188,
  60.03094335339014,
  78.92870155469258,
  71.44919290222474,
  34.26440692694148,
  55.350452392555816,
  62.442061971394416,
  49.05377350402926,
  53.989395547798324,
  76.25958585567928,
  85.7899115297591,
  65.47074950213768,
  51.26830506720621,
  68.40707017600195,
  37.90070017935406,
  36.110559806212045,
  73.536271953554,
  70.6686371

### This code can be used to produce binary DES from the list of volumes and cocnentrated stock solutions of your components. Capable of switching between two different pipettes, but modify as necessary. Code is originally from Newcastle IGEM at Newcastle University, github can be found here https://github.com/jbird1223/Newcastle-iGEM

In [13]:
from opentrons import labware, instruments, robot
from sqlite3 import IntegrityError
robot.reset() #remove this when uplaoding code to robot

#####################################################################################################################################################
#####################################################################################################################################################		

# Import Labware

tiprack_300 = labware.load("opentrons-tiprack-300ul", '1')      
tiprack_1000 = labware.load("tiprack-1000ul", '4')                
Stock1 = labware.load("opentrons-tuberack-15ml", '2')                                           
wellplate_96 = labware.load("96-flat", '3')
trash = robot.fixed_trash

#####################################################################################################################################################
#####################################################################################################################################################

# Import Pipettes

P1000 = instruments.P1000_Single(
    mount = 'right',
    tip_racks = [tiprack_1000],
    trash_container = trash
)

P300 = instruments.P300_Single(
    mount='left',
    tip_racks=[tiprack_300],
    trash_container=trash 
)

#####################################################################################################################################################
#####################################################################################################################################################

#Insert DES Generator here. 

reagents1 = convert_mole_fractions_to_volumes(3, 4, 0.3, 0.2, 0.8, 0.7, 96, 192 )

reagent_pos1 = ['A1', 'A3']



#####################################################################################################################################################
#####################################################################################################################################################
# STOCK LABWARE 1

robot.home()                                                    # Homes robot and prevents any pipette bugs 
for counter, reagent in enumerate(reagents1,0):
                                                                # These objects are temporary and will only exist within this loop
    source      = reagent_pos1[counter]                         # Counter is use to index an independent list (e.g. reagent_pos)
    P1000list   = [source]                                      # This is then added to both list - used in testing 				
    P300listn   = [source]


    P300.pick_up_tip()                                          # Picks up pipette tip for both P10 and P300 to allow to alternate
    P1000.pick_up_tip()
    for well_counter, values in enumerate(reagent):             # Specifies the well position and the volume of reagent being 
        if values == float(0):                                  # If volume is 0, well is skipped  
            pass
        elif values < float(300):
            P300.distribute(                                     # If volume below 300, P300 used not p1000. Greater than 300 P1000 is used.
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),          # Prevents submerging tip in solution, not completely sterile, but beneficial
                blow_out=True,                                  # Removes excess from tip
                rate=1,                                         # How quick it aspirates/dispenses, lower (ie 0.5) if stock viscous
                new_tip='never')
            P300.touch_tip(wellplate_96(well_counter))         # Touches tip to remove any droplets
            P300.blow_out(wellplate_96(well_counter))
        else: 
            P1000.distribute(
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),
                blow_out=True,
                rate=1,
                new_tip='never')
            P1000.touch_tip(wellplate_96(well_counter))
            P1000.blow_out(wellplate_96(well_counter))
    P1000.drop_tip()
    P300.drop_tip()

for c in robot.commands(): # remove this when uploading code to robot
     print(c)


ModuleNotFoundError: No module named 'opentrons'